<a href="https://colab.research.google.com/github/lbhagavan/stanford_LLM_Leela/blob/homework/Carlos_LLM_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install

In [ ]:
# OpenAI - Llama Index - Langchain
!pip install openai
!pip install sentence-transformers
!pip install langchain pypdf langchain-openai
!pip install llama-index --upgrade

# SubQuery
!pip install nest-asyncio

# CrewAI - DuckDuckGo - GoogleGenAI
%pip install --q crewai
%pip install --q -U duckduckgo-search
%pip install --q langchain_google_genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 14.4 MB/s e

##Get Data

In [ ]:
#!wget https://static.www.nfl.com/image/upload/league/apps/league-site/media-guides/2022/2022_NFL_Record_and_Fact_Book.pdf?cid=nfl-history
!wget https://operations.nfl.com/media/hfjn4oid/2024-record-fact-book-incl-supplemental.pdf

--2024-08-27 00:53:10--  https://operations.nfl.com/media/hfjn4oid/2024-record-fact-book-incl-supplemental.pdf
Resolving operations.nfl.com (operations.nfl.com)... 151.101.1.153, 151.101.65.153, 151.101.129.153, ...
Connecting to operations.nfl.com (operations.nfl.com)|151.101.1.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27181782 (26M) [application/pdf]
Saving to: ‘2024-record-fact-book-incl-supplemental.pdf’

2024-record-fact-bo 100%[===================>]  25.92M   166MB/s    in 0.2s    

2024-08-27 00:53:12 (166 MB/s) - ‘2024-record-fact-book-incl-supplemental.pdf’ saved [27181782/27181782]



## Imports & Config

In [ ]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('api_key')
client = OpenAI(api_key=open_ai_key)

import os
os.environ["OPENAI_API_KEY"] = open_ai_key

# Import necessary classes from the llama_index package
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, SummaryIndex
from llama_index.core import Settings


##Read Data

In [ ]:
# Combine the data
documents = SimpleDirectoryReader("./").load_data("2024-record-fact-book-incl-supplemental.pdf")


# Read documents from the specified directory and load a specific document, "report.pdf".
# documents = SimpleDirectoryReader("./final_project").load_data("*.pdf")
# reader1 = SimpleDirectoryReader("./final_project/bets")
# reader2 = SimpleDirectoryReader("./final_project/docs")
# reader3 = SimpleDirectoryReader("./final_project/teams")

# Load data from each directory
# bets = reader1.load_data("*.pdf")
# docs = reader2.load_data("*.pdf")
# teams = reader3.load_data("*.pdf")



Loading files: 100%|██████████| 1/1 [02:08<00:00, 128.18s/file]


##RAG & Chat

In [ ]:
# Combine the data
documents = SimpleDirectoryReader("./").load_data("2024-record-fact-book-incl-supplemental.pdf")
# bets + docs + teams

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
index = VectorStoreIndex.from_documents(documents)

# Convert the VectorStoreIndex object into a query engine. This query engine can be used to
# perform semantic searches on the index, matching natural language queries to the most relevant
# documents in the index.
query_engine = index.as_query_engine()

# Use the query engine to search for documents that are relevant to the query
# from the indexed documents based on the semantic understanding of the query.
response = query_engine.query("Is 49ers a team in NFL?")

# chat_engine to test chat functionality
chat_engine = index.as_chat_engine(chat_mode="openai", tool_choice="query_engine_tool", verbose=False)

# initialize settings (set chunk size)
Settings.chunk_size = 1024
nodes = Settings.node_parser.get_nodes_from_documents(documents)

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

Loading files: 100%|██████████| 1/1 [02:12<00:00, 132.86s/file]


###Tests RAG

In [ ]:
# Print the response obtained from the query. This will display the result of the semantic search,
# showing the information or documents that best match the query about the 2024 outlook.
response = query_engine.query("Does the 49ers the best team in NFC West?")
print("RAG => Does the 49ers the best team in NFC West?")
print(response)
print("==========")

response = query_engine.query("Which is the team that the 49ers beat more in NFL?")
print("RAG => Which is the team that the 49ers beat more in the NFL?")
print(response)
print("==========")

response = query_engine.query("Is the 49ers favorite to win NFC this season?")
print("RAG => Which is the best NFL team based on the current data?")
print(response)
print("==========")


RAG => Does the 49ers the best team in NFC West?
The 49ers are one of the teams in the NFC West, but based on the context provided, it does not explicitly state whether they are the best team in the NFC West.
RAG => Which is the team that the 49ers beat more in the NFL?
The team that the 49ers beat more in the NFL is the Washington Redskins.
RAG => Which is the best NFL team based on the current data?
The context does not provide any information about the current season or the favorites to win the NFC.


###Tests Chat

In [ ]:
#Print the response obtained from the chat.
response = chat_engine.chat(
    "Does the 49ers the best team in NFC West?"
)
print("CHAT => Does the 49ers the best team in NFC West?")
print(response)
print("==========")

response = chat_engine.chat("Is the 49ers favorite to win NFC this season?")
print("CHAT => Is the 49ers favorite to win NFC this season?")
print(response)
print("==========")

response = chat_engine.chat("Which is the team that the 49ers beat more in NFL?")
print("CHAT => Which is the team that the 49ers beat more in NFL?")
print(response)
print("==========")

response = chat_engine.chat(
    "Which is the best NFL team based on the current data?")
print("CHAT => Which is the best NFL team based on the current data?")
print(response)
print("==========")


CHAT => Does the 49ers the best team in NFC West?
Based on the information available, it is not explicitly stated whether the 49ers are the best team in the NFC West.
CHAT => Is the 49ers favorite to win NFC this season?
The 49ers are not the favorite to win the NFC this season.
CHAT => Which is the team that the 49ers beat more in NFL?
The 49ers have beaten the Rams the most in the NFL.
CHAT => Which is the best NFL team based on the current data?
Based on the current data provided, the best NFL team would be the Denver Broncos as they won the Super Bowl XXXII against the Green Bay Packers with a score of 31-24.


##Sub Question Query Engine

In [ ]:
# Import necessary classes and modules from llama_index.core and llama_index.core.tools
import nest_asyncio
nest_asyncio.apply()

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core import Settings
from IPython.display import display, Markdown

# Build a VectorStoreIndex from the loaded documents
# This index will allow efficient querying of the document content
vector_query_engine = VectorStoreIndex.from_documents(
    documents,
    use_async=True,  # Enable asynchronous processing for faster performance
).as_query_engine()  # Convert the index to a query engine

# Define a list of query engine tools, each with its own metadata
# This setup is necessary for the SubQuestionQueryEngine
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,  # Use the vector_query_engine built above
        metadata=ToolMetadata(
            name="documents",  # Name of the tool
            description="Report",  # Description of the tool
        ),
    ),
]

# Create an instance of SubQuestionQueryEngine using the default settings
# This query engine can handle sub-questions and use the provided tools for querying
query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,  # Provide the list of query engine tools
    use_async=True,  # Enable asynchronous processing
)

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:193: RuntimeWarning: coroutine 'run_async_tasks.<locals>._gather' was never awaited
  self.__pydantic_validator__.validate_python(data, self_instance=self)
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:193: RuntimeWarning: coroutine 'SubQuestionQueryEngine._aquery_subq' was never awaited
  self.__pydantic_validator__.validate_python(data, self_instance=self)
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:193: RuntimeWarning: coroutine 'VectorStoreIndex._async_add_nodes_to_index' was never awaited
  self.__pydantic_validator__.validate_python(data, self_instance=self)


###Tests Sub Question Query

In [ ]:
response = query_engine.query(
    "Does the 49ers the best team in NFC West?"
)
print("SUBQUERY => Does the 49ers the best team in NFC West?")
print(response.response)
print("==========")

response = query_engine.query(
    "Which is the team that the 49ers beat more in NFL?"
)
print("SUBQUERY => Which is the team that the 49ers beat more in NFL?")
print(response.response)
print("==========")

response = query_engine.query(
    "Which is the best NFL team based on the current data?"
)
print("SUBQUERY => Which is the best NFL team based on the current data?")
print(response.response)
print("==========")


Generated 1 sub questions.
[documents] Q: What is the current standing of the 49ers in the NFC West?
[documents] A: The San Francisco 49ers currently have a standing of 114 wins and 96 losses in the NFC West.
SUBQUERY => Does the 49ers the best team in NFC West?
The San Francisco 49ers may not necessarily be the best team in the NFC West based on their current standing of 114 wins and 96 losses in the division.
Generated 1 sub questions.
[documents] Q: List all the teams that the 49ers have beaten in NFL
[documents] A: The teams that the 49ers have beaten in the NFL based on the provided context are the Bears, Rams, and Steelers.
SUBQUERY => Which is the team that the 49ers beat more in NFL?
The team that the 49ers have beaten more in the NFL based on the provided context is the Rams.
Generated 2 sub questions.
[documents] Q: What is the current ranking of NFL teams?
[documents] Q: What are the recent performance statistics of NFL teams?
[documents] A: The recent performance statistics

##Agents

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import OpenAI
from crewai import Agent, Task, Crew, Process
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

llm = OpenAI(temperature=0.1, openai_api_key=userdata.get('api_key'))

def getAgentsTasks(question_to_answer, verbose):
  # Define your agents with roles and goals
  researcher = Agent(
    role='Senior Research Analyst',
    goal=f'Conduct comprehensive research to answer the question {question_to_answer}. Give more weight in the last 2 seasons, where 49ers have same players and tendencies as the other NFC West teams.',
    backstory="""You are getting all data and analytics from NFL history.
    You have focused your career in 49ers data.
    You need to create forecasts for every game based on the last games.""",
    verbose=verbose,
    allow_delegation=True,
    llm = llm,
    tools=[search_tool]
  )

  critic = Agent(
    role='Critic',
    goal='Criticize output of research analysts to ensure the content is really insightful and not generic',
    backstory="""You are a senior NFL analyst, you have accurate of more than 75% of 49ers game forecasts since last 2 years""",
    verbose=verbose,
    allow_delegation=False,
    llm = llm,
    tools=[]
  )

  writer = Agent(
    role='Responder',
    goal=f'Understands user questions and can answer the main question: {question_to_answer}',
    backstory="""You understand users that need to know about 49ers games.
    You want to understand them and give the best accurate answers from their questions.""",
    verbose=verbose,
    allow_delegation=False,
    llm = llm,
    tools=[]
  )

  # Create tasks for your agents
  task1 = Task(
    description=f"""Conduct a comprehensive research analysis to answer the question: {question_to_answer}""",
    agent=researcher,
    expected_output=f"Conduct a comprehensive research analysis to answer the question: {question_to_answer}"  # Add the expected output field
  )

  task2 = Task(
    description="""Critique the report and insights generated""",
    agent=critic,
    expected_output="A set of criticisms and suggested improvements"  # Add the expected output field
  )

  task3 = Task(
    description=f"""Using the insights provided, develop a research analysis to answer the question: {question_to_answer}""",
    agent=writer,
    expected_output=f"A full research report of 1 paragraph research analysis answering the question: {question_to_answer}." # Added expected output
  )

  return {
      'agents': [researcher, critic, writer],
      'tasks': [task1, task2, task3]
  }

def getCrewWork(agents, tasks, verbose):
  # Instantiate your crew with a sequential process
  crew = Crew(
    agents=agents,
    tasks=tasks,
    verbose=verbose, # You can set it to 1 or 2 to different logging levels,
  )

  # Get your crew to work!
  result = crew.kickoff()

  return result

In [ ]:
question_to_answer = "Does the 49ers the best team in NFC West?"

info = getAgentsTasks(question_to_answer, False)
result = getCrewWork(info['agents'], info['tasks'], False)
print("AGENTS => Does the 49ers the best team in NFC West?")
print(result)
print("==========")

question_to_answer = "Which is the team that the 49ers beat more in NFL?"
info = getAgentsTasks(question_to_answer, False)
result = getCrewWork(info['agents'], info['tasks'], False)
print("AGENTS => Which is the team that the 49ers beat more in NFL?")
print(result)
print("==========")

question_to_answer = "Which is the best NFL team based on the current data?"
info = getAgentsTasks(question_to_answer, False)
result = getCrewWork(info['agents'], info['tasks'], False)
print("AGENTS => Which is the best NFL team based on the current data?")
print(result)
print("==========")


AGENTS => Does the 49ers the best team in NFC West?
Based on the research conducted using the duckduckgo_search tool, it is evident that the San Francisco 49ers have been a dominant team in the NFC West over the past 2 seasons. However, a thorough analysis of the team's performance is necessary to determine if they are truly the best team in the division. While the report mentions that the 49ers have won the division title in 3 out of the last 5 seasons, it fails to provide specific details on the seasons in which they achieved this feat. Additionally, while their record has improved to 11-0 in the last 2 seasons, there is no comparison to their record in previous seasons, making it difficult to determine the extent of their improvement. Furthermore, the report mentions key players such as Brock Purdy and Christian McCaffrey as MVP candidates, but lacks analysis and statistics to support this claim. A more comprehensive analysis of the team's performance and comparison to other teams i

AGENTS => Which is the team that the 49ers beat more in NFL?
After conducting thorough research and analysis, it can be concluded that the team that the 49ers have beaten the most in the NFL is the Green Bay Packers. However, there are some limitations and areas for improvement in the current report and insights. Firstly, the report lacks context as it does not provide details on the timeframe or notable events that may have contributed to the 49ers' success against the Packers. Additionally, the scope of the research is limited to only the last two seasons, which may not accurately reflect the overall history of the two teams. Furthermore, the data is incomplete as it only includes the 49ers' record against the Packers in the playoffs and regular season, without considering other games such as preseason or primetime matchups. Lastly, the report lacks comparison to other teams, which could provide a more comprehensive understanding of the 49ers' success against the Packers. In order to